## READ TEST, TRAIND AND VALIDATION FILES

In [1]:
import pandas as pd
# Read the .tsv file into a Pandas DataFrame
df_train = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/train.tsv', sep='\t')
df_valid = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/validation.tsv', sep='\t')
df_test = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/test.tsv', sep='\t')
# Print the first few rows of the DataFrame

In [2]:
df_train[0]

,file_name,label,shaming,stereotype,objectification,violence,text
0,8716.jpg,1,0,1,1,0,GETS MARRIED TO GIRL OF HIS DREAMS SHE DOESN'T...
1,3066.jpg,1,1,1,0,0,When your mama don't change yo diaper for 19 y...
2,6038.jpg,0,0,0,0,0,"Some people want a big house, a fast car, and ..."
3,10861.jpg,1,0,0,1,0,FAP FAP FAP FAP FAP FAP memecenter Meme Center
4,11198.jpg,0,0,0,0,0,I RAISE. A I CALL. I FOLD. I'M ALL IN
...,...,...,...,...,...,...,...
8995,8010.jpg,1,0,0,1,0,A good girlfriend can save 200GB of your HDD N...
8996,10357.jpg,1,0,0,1,1,TONIGHT I WILL CUM TOMORROW HE WILL GO TO JAIL...
8997,3495.jpg,1,0,1,1,1,WOMEN WHO DATE AND MARRY FOR MONEY ARE JUST MO...
8998,7304.jpg,0,0,0,0,0,MOVIES TODAY WHO THE FUCK CARES A TROLL COMPAN...


### Attaching the image file_path

In [3]:
import pandas as pd

# Read the .tsv file into a Pandas DataFrame
df_train = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/train.tsv', sep='\t')
df_valid = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/validation.tsv', sep='\t')
df_test = pd.read_csv('/kaggle/input/multimodal-misogyny-detection-mami-2022/test.tsv', sep='\t')

# Define the path to the image folders
train_image_folder = '/kaggle/input/multimodal-misogyny-detection-mami-2022/MAMI_2022_images/training_images/'
test_image_folder = '/kaggle/input/multimodal-misogyny-detection-mami-2022/MAMI_2022_images/test_images/'

# Function to generate file path using file name
def get_file_path(row):
    if row['file_name'] in df_train['file_name'].values:
        return train_image_folder + row['file_name']
    elif row['file_name'] in df_valid['file_name'].values:
        return train_image_folder + row['file_name']
    elif row['file_name'] in df_test['file_name'].values:
        return test_image_folder + row['file_name']
    else:
        return None

# Apply the function to generate file path
df_train['file_path'] = df_train.apply(get_file_path, axis=1)
df_valid['file_path'] = df_valid.apply(get_file_path, axis=1)
df_test['file_path'] = df_test.apply(get_file_path, axis=1)

In [4]:
df_valid.file_path[0]

'/kaggle/input/multimodal-misogyny-detection-mami-2022/MAMI_2022_images/training_images/1377.jpg'

### RUNNING AN INFERENCE ON THE TEST IMAGES using nlpconnect/vit-gpt2-image-captioning
This is to check the model behave on these images

In [8]:
import pandas as pd
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from PIL import Image

# Load pre-trained model and tokenizer
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Move model to available device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set generation parameters
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

# Define function to predict captions for images with attention mask
def predict_caption(df):
    captions = []
    for index, row in df.iterrows():
        image_path = row['file_path']
        image = Image.open(image_path)
        if image.mode != "RGB":
            image = image.convert(mode="RGB")
        
        pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        
        # Create attention mask to avoid padding tokens affecting the output
        attention_mask = torch.ones_like(pixel_values)
        attention_mask[pixel_values == 0] = 0
        
        output_ids = model.generate(pixel_values, attention_mask=attention_mask, **gen_kwargs)
        preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        preds = [pred.strip() for pred in preds]
        
        captions.append(preds[0])  # Assuming only one caption per image
        
    return captions

Predict and plot only 5 random images


In [ ]:
from PIL import Image
import random
import matplotlib.pyplot as plt

# Select 5 random images from the test set
random_indices = random.sample(range(len(df_test)), 5)
df_sampled = df_test.iloc[random_indices]

# Predict captions for the sampled images
predicted_captions_sampled = predict_caption(df_sampled)

# Plot the sampled images with their predicted captions
plt.figure(figsize=(15, 10))
for i, (index, row) in enumerate(df_sampled.iterrows()):
    image_path = row['file_path']
    image = Image.open(image_path)
    
    plt.subplot(2, 3, i+1)
    plt.imshow(image)
    plt.title(predicted_captions_sampled[i])
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Print the first few rows of the DataFrame with predicted captions
df_pred.caption[6]

### USING OCR TO REAF THE TEXT IN THE IMAGES